In [1]:
import gensim
import os
import collections
import smart_open
import random

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

In [4]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [5]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [2]:
# my training data
filename = "C:/my_temp/abstracts_train.csv"
lines = tuple(open(filename, 'r'))

train_corpus = []
for i, line in enumerate(lines):
    x = gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])
    train_corpus.append(x)

filename = "C:/my_temp/abstracts_test.csv"
lines = tuple(open(filename, 'r'))

test_corpus = []
for i, line in enumerate(lines):
    x = gensim.utils.simple_preprocess(line)
    test_corpus.append(x)
print(len(train_corpus),len(test_corpus))

207 90


In [5]:
lines[0]

'NK cells have two main functions , namely cell-mediated cytotoxicity and production of cytokines . Multiple inhibitory receptors that regulate NK-cell cytotoxicity have been characterized whereas little is known about receptors regulating cytokine production . Here we report that CD72 , which is considered to be an important co-receptor regulating B-cell activation , is also expressed on mouse NK cells . NK cells expressing high levels of CD72 , upon stimulation with IL-12 and IL-18 or target cells , produce significantly less IFN-gamma than those expressing low levels of CD72 , whereas both subsets are equally cytotoxic . Ectopic expression of CD72 in the murine NK-cell line KY2 inhibits cytokine-induced IFN-gamma production , and the inhibitory effect is diminished by mutations in the inhibitory motifs in the intracellular domain or replacement of the extracellular domain of CD72 . Thus , CD72 is an inhibitory receptor on NK cells regulating cytokine production .\n'

In [59]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=55)
model.build_vocab(train_corpus)
#%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [47]:
type(train_corpus)
#gensim.models.doc2vec.TaggedDocument?

list

In [60]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 0.08906716,  0.07031029, -0.02158041,  0.07074333, -0.08068049,
        0.0936869 , -0.15466854, -0.04996001,  0.11892247,  0.06536631,
       -0.02590953,  0.19045809, -0.04241417, -0.09479586, -0.13978249,
       -0.09453529, -0.05565806,  0.11436072, -0.10992189, -0.0530768 ,
        0.03504281,  0.05017278,  0.06948558, -0.02832355,  0.01683649,
        0.08904222, -0.06829819, -0.06163589, -0.08909246, -0.03931525,
        0.00189355, -0.06749137,  0.00421021,  0.08870339, -0.07675441,
       -0.07545242,  0.06011757,  0.0799645 ,  0.1343765 ,  0.08767392,
       -0.09484342, -0.11994749,  0.0630477 , -0.03426826, -0.01381412,
        0.1109122 ,  0.01133253, -0.14219338, -0.07205504, -0.00281544],
      dtype=float32)

In [34]:
# Pick a random document from the test corpus and infer a vector from the model
sc=0
max_sc = sc
trash=0.9
mintrash=0.8
while sc<trash:
    doc_id = random.randint(0, len(test_corpus) - 1)
    inferred_vector = model.infer_vector(test_corpus[doc_id])
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    sc = sims[0][1]
    max_sc = max(max_sc,sc)
    if max_sc==sc and max_sc> mintrash:        
        # Compare and print the most/median/least similar documents from the train corpus
        print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
        print(sims[0][1], ' '.join(train_corpus[sims[index][0]].words),"\n\n") 

#, ' '.join(train_corpus[sims[0][0]].words))
#print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
#for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    #print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (30): «police are combing through videotapes trying to spot the gunman dressed in black who shot year old man to death at downtown massage parlour the victim was hit in the stomach and upper body and died about hours later in hospital the woman was not hurt police urged business owners to turn over any security camera videotapes they might have that recorded people on the street at the time several such videos are now being reviewed»

0.843387246131897 federal treasurer peter costello has warned continued economic growth in australia is dependent on an uncertain world outlook the latest figures show the economy grew by per cent in the september quarter mr costello is stressing the seriousness of the current global economic downturn as serious as anything we ve seen in the last two decades he said both he and the reserve bank governor ian macfarlane believe the timing of any pickup is unclear mr costello is hopeful about the united states prospects next year and says picku

KeyboardInterrupt: 

In [69]:
# Pick a random document from the test corpus and infer a vector from the model
sc=0
max_sc = sc
trash=0.95
mintrash=0.88
while sc<trash:
    doc_id = random.randint(0, len(test_corpus) - 1)    
    #print(type(test_corpus[doc_id]))
    inferred_vector = model.infer_vector(test_corpus[doc_id])
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    sc = sims[0][1]
    max_sc = max(max_sc,sc)
    if max_sc==sc and max_sc> mintrash:        
        # Compare and print the most/median/least similar documents from the train corpus
        print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
        print(sims[0])
        print(' '.join(train_corpus[sims[0][0]].words),"\n\n") 

#, ' '.join(train_corpus[sims[0][0]].words))
#print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
#for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    #print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (69): «background the of hiv specific cell responses in people infected with locally circulating hiv strain will facilitate the development of hiv vaccine sixty intravenous drug users infected with hiv circulating recombinant form crf which has been spreading rapidly in western china from north to south were recruited from xinjiang china to assess the hiv specific cell responses at single peptide level with overlapping peptides olp covering the whole concensus clades and proteome results the median of the total magnitude and total number of olps recognized by ctl responses were sfc million pbmc and olps respectively when tested by clade peptides which was significantly higher than when tested by clade peptides the immunodominant regions which cover of the hiv proteome are widely distributed throughout the hiv proteome except in tat vpu and pol pr with gag pol rt pol int and nef being most frequently targeted the subdominant epitopes are mostly located in nef integrase vpr

Test Document (82): «two years ago ce certified interferon gamma release assays igra were launched on the german market quantiferon tb gold in tube and spot tb since this time multitude of studies have analysed these assays guidelines have been elaborated by national expert committees of england the usa and switzerland however standards of tuberculosis diagnostics and management may vary from country to country this statement provides practice relevant recommendations for indications pre analytics and the interpretation of igra test results under different clinical conditions the igra are integrated into existing guidelines for the management of tuberculosis»

(142, 0.904216468334198)
two new tests quantiferon tb and spot tb for diagnosis of latent tuberculosis are on the market they measure the release of interferon gamma in whole blood in response to stimulation by pdd they offer more accurate approach than tuberculin skin test for identification of individuals with latent tuberculos

KeyboardInterrupt: 